In [ ]:
import torch
import pandas as pd
import numpy as np
import importlib
import ModelRunner as MR
import MakeDataset as MD
import SiameseNeuralNetwork as SNN
import SimpleNeuralNetwork as SimpNN
from matplotlib import pyplot as plt
from pubchempy import *
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import PeriodicTable as PT
import re

In [ ]:
# Load the dataset. Clean it up by removing missing molecule features. 
# molecule_features: First col is a string containing name of molecule. Rest are floats containing its features
# eutectic_compilation: First 2 cols are strings containing molecule names, third col is eutectic proportion, fourth is eutectic temperature
molecule_features = pd.read_csv("D:\\Research\\UConn_ML\\data\\eutectic_mixtures-main\\single_components.csv").drop(["xlogp", "rotatable_bond_count"], axis=1)
eutectic_compilation = pd.read_csv("D:\\Research\\UConn_ML\\data\\eutectic_mixtures-main\\eutectic_compilation.csv")

molecule_features[molecule_features.columns[1:]] = molecule_features[molecule_features.columns[1:]].astype(float)
eutectic_compilation[eutectic_compilation.columns[3]] = eutectic_compilation[eutectic_compilation.columns[3]].astype(float)

In [ ]:
# Some indices in eutectic_compilation do not exist in single_components. They will need to be removed.
# Some of the eutectic proportions are inconvertible to floats from strings. They will also need to be removed.
# missing_molecules will store all molecules that are missing features from eutectic_compilation
drops = np.array([]).astype(int)
missing_molecules = np.array([]).astype(str)
for i in range(len(eutectic_compilation)):
    ec = eutectic_compilation.iloc[i]
    m1 = ec[0]
    m2 = ec[1]
    xe = ec[2]
    
    m1f = molecule_features.loc[molecule_features.mol == ec[0]]
    m2f = molecule_features.loc[molecule_features.mol == ec[1]]
    
    try:
        xe = float(xe)
    except ValueError:
        drops = np.append(drops, i)
    
    if(len(m1f) == 0 or len(m2f) == 0):
        drops = np.append(drops, i)
        
        if(len(m1f) == 0):
            missing_molecules = np.append(missing_molecules, m1)
        elif(len(m2f) == 0):
            missing_molecules = np.append(missing_molecules, m2)

eutectic_compilation = eutectic_compilation.drop(eutectic_compilation.index[drops])
# np.savetxt("missing_molecules.csv", missing_molecules, delimiter=",", fmt="%s")

xe = torch.tensor(np.array(eutectic_compilation.xe).astype(float) / 100.0)
eutectic_compilation.xe = xe

In [ ]:
eutectic_compilation = eutectic_compilation.drop_duplicates(subset=['molA', 'molB'])

In [ ]:
# Split the eutectic compilation dataframe by a ratio into training and testing sets
split = 0.90
train_ec = eutectic_compilation.sample(frac=split)
test_ec = eutectic_compilation.drop(train_ec.index)

split = 0.15
val_ec = train_ec.sample(frac=split)
train_ec = train_ec.drop(val_ec.index)

In [ ]:
# Obtain the indices of train, val, and test datasets in molecule_features
trainindices = np.array([]).astype(int)
valindices = np.array([]).astype(int)
testindices = np.array([]).astype(int)

for i in range(len(train_ec)):
    line = train_ec.iloc[i]
    
    m1 = line[0]
    m2 = line[1]
    
    trainindices = np.append(trainindices, np.where(molecule_features.mol == m1))
    trainindices = np.append(trainindices, np.where(molecule_features.mol == m2))

for i in range(len(val_ec)):
    line = val_ec.iloc[i]
    
    m1 = line[0]
    m2 = line[1]
    
    valindices = np.append(valindices, np.where(molecule_features.mol == m1))
    valindices = np.append(valindices, np.where(molecule_features.mol == m2))
    
for i in range(len(test_ec)):
    line = test_ec.iloc[i]
    
    m1 = line[0]
    m2 = line[1]
    
    testindices = np.append(testindices, np.where(molecule_features.mol == m1))
    testindices = np.append(testindices, np.where(molecule_features.mol == m2))
    
trainindices = np.unique(trainindices)
valindices = np.unique(valindices)
testindices = np.unique(testindices)

In [ ]:
address = ['n1', 'c1', 'n2', 'c2', 'n3', 'c3', 'n4', 'c4', 'n5', 'c5']
for i in range(len(address)):
    molecule_features[address[i]] = np.zeros((len(molecule_features)))

In [ ]:
# Add 8 more features using PubChemPy and test with those. In notepad file from meeting. 
importlib.reload(PT)
pt = PT.PT()

for m in range(len(molecule_features.mol)):
    split = re.findall('[A-Z][^A-Z]*', re.sub('[^\w]', "", molecule_features.mol[m])) 
    # split the molecule by elements (with count of each element) and remove any symbols (only letters and numbers)
    
    nummolecules = len(split) # obtain number of elements after split
    atomic_nums = np.array([])
    atomic_counts = np.array([])
    masses = np.array([])
    
    for s in range(len(split)):
        formula = re.sub('[^A-Za-z]', "", split[s]) # obtain each element alone
        count = re.sub('[A-Za-z]', "", split[s]) # obtain the count of each element alone
        
        if count == "":
            count = "1"
        count = int(count) # convert the string to int
        
        # Obtain the atomic number and mass by using the formula from periodic table. 
        data = pt.get(formula)
        atomic_nums = np.append(atomic_nums, data[0])
        masses = np.append(masses, data[1])
        atomic_counts = np.append(atomic_counts, count)
        
        # print(formula, count, type(formula), type(count)) # to see what prints
        
    # Sort in ascending order, the atomic numbers and counts by mass of atom
    atomic_nums = [atomic_nums for _, atomic_nums in sorted(zip(masses, atomic_nums))]
    atomic_counts = [atomic_counts for _, atomic_counts in sorted(zip(masses, atomic_counts))]
    
    # Add into the molecule_features dataframe in correct order
    i = 0
    j = 0
    for i in range(len(atomic_nums)):
        molecule_features[molecule_features.columns[j+5]][m] = atomic_nums[i]
        molecule_features[molecule_features.columns[j+6]][m] = atomic_counts[i]
        j += 2

In [ ]:
fig0, axes = plt.subplots(len(molecule_features.columns) - 1, 1)
fig0.set_figheight(50)
fig0.set_figwidth(15)

for i in range(len(molecule_features.columns) - 1):
    axes[i].boxplot(molecule_features[molecule_features.columns[i+1]], vert=False)
    axes[i].set(ylabel=molecule_features.columns[i+1])

fig0.savefig('D:\\Research\\UConn_ML\\Plots\\Feature_Box_Plots_8_17_22.png')

In [ ]:
molecule_features

In [ ]:
# Z-Score Scaler transformation on molecule features excluding new added features since they are almost categorical
# scaler = StandardScaler()
# scalecols = [1, 2, 3, 4, 5, 7, 9, 11, 13]
# molecule_features.iloc[trainindices, scalecols] = scaler.fit_transform(molecule_features.iloc[trainindices, scalecols])
# molecule_features.iloc[valindices, scalecols] = scaler.transform(molecule_features.iloc[valindices, scalecols])
# molecule_features.iloc[testindices, scalecols] = scaler.transform(molecule_features.iloc[testindices, scalecols])

In [ ]:
molecule_features

In [ ]:
eutectic_compilation

In [ ]:
train_ec

In [ ]:
val_ec

In [ ]:
test_ec

In [ ]:
print("Train Size: ", len(train_ec))
print("Validation Size: ", len(val_ec))
print("Test Size: ", len(test_ec))

In [ ]:
importlib.reload(MD)

In [ ]:
# Define parameters and datasets to pass into trainer. Pass in order of definition
starting_features = 14
batchsize = 100
max_epochs = 15
lrbase = 1e-6
lrmax = 1e-3

# For xe
overfit_bound = 0.25
# For Te
# overfit_bound = 0.3

train_dset = MD.MD(train_ec, molecule_features, starting_features)
val_dset = MD.MD(val_ec, molecule_features, starting_features)
test_dset = MD.MD(test_ec, molecule_features, starting_features)

# The best constant value to use for MAE is the median of the dataset
# MAE tells us how much + or - pred is from truth on average
# train_constant = torch.std(torch.tensor(np.array(train_ec["xe"]).astype(float)))**2
train_constant = torch.mean(torch.tensor(np.array(train_ec["Te"]).astype(float)))

train_param = "Te"

In [ ]:
importlib.reload(MR)
importlib.reload(SNN)
importlib.reload(SimpNN)
modelrunner = MR.MR(starting_features, batchsize, max_epochs, lrbase, lrmax,
                    train_dset, val_dset, test_dset, train_constant, train_param)

In [ ]:
trloss, trbase, vloss, vbase = modelrunner.train_and_validate('siam')

In [ ]:
outputs, invouts, truths, testloss, r2 = modelrunner.test()

In [ ]:
simptrloss, simptrbase, simpvloss, simpvbase = modelrunner.train_and_validate('simp')

In [ ]:
simpoutputs, simpinvouts, simptruths, simptestloss, simpr2 = modelrunner.test()

In [ ]:
l1 = len(trloss)
l2 = len(simptrloss)
x1 = np.arange(l1)
x2 = np.arange(l2)

lossfig, axes = plt.subplots(2, 2)
lossfig.set_figheight(15)
lossfig.set_figwidth(15)
            
axes[0, 0].plot(x1, trloss[0:l1], label="Train Running Loss", c="blue")
axes[0, 0].plot(x1, trbase[0:l1], label="Train Baseline Loss", c="red")
axes[0, 0].set_title("Graph of Siamese NN Training Loss Against a Baseline")
axes[0, 0].set(xlabel="Epoch", ylabel="Siamese MAE Loss")
axes[0, 0].legend(loc="upper right")

axes[0, 1].plot(x1, vloss[0:l1], label="Val Running Loss", c="blue")
axes[0, 1].plot(x1, vbase[0:l1], label="Val Baseline Loss", c="red")
axes[0, 1].set_title("Graph of Siamese NN Validation Loss Against a Baseline")
axes[0, 1].set(xlabel="Epoch", ylabel="Siamese MAE Loss")
axes[0, 1].legend(loc="upper right")

axes[1, 0].plot(x2, simptrloss[0:l2], label="Train Running Loss", c="blue")
axes[1, 0].plot(x2, simptrbase[0:l2], label="Train Baseline Loss", c="red")
axes[1, 0].set_title("Graph of Simple NN Training Loss Against a Baseline")
axes[1, 0].set(xlabel="Epoch", ylabel="Simple MAE Loss")
axes[1, 0].legend(loc="upper right")

axes[1, 1].plot(x2, simpvloss[0:l2], label="Val Running Loss", c="blue")
axes[1, 1].plot(x2, simpvbase[0:l2], label="Val Baseline Loss", c="red")
axes[1, 1].set_title("Graph of Simple NN Validation Loss Against a Baseline")
axes[1, 1].set(xlabel="Epoch", ylabel="Simple MAE Loss")
axes[1, 1].legend(loc="upper right")

plt.show()

In [ ]:
l = batchsize
numplots = int(len(outputs) / l)
siamfig, axes = plt.subplots(numplots, 2)
siamfig.set_figheight(40)
siamfig.set_figwidth(15)

pred = 0
succ = 1
x = np.arange(l)
acc = np.linspace(0, 1, 10)
mae = np.round(testloss, 3)
        
for row in range(numplots):        
    axes[row, 0].scatter(x, outputs[pred*l:succ*l] - truths[pred*l:succ*l], c="red")
    axes[row, 0].plot(x, np.zeros((l,)), c="green", label="0 Point")
    axes[row, 0].set(xlabel="Data Points", ylabel="Residuals")
    axes[row, 0].legend(loc="upper right")

    axes[row, 1].scatter(truths[pred*l:succ*l], outputs[pred*l:succ*l], c="green")
    axes[row, 1].plot(acc, acc, label="Accuracy Line")
    
    if train_param == "xe":
        axes[row, 1].set(xlabel="Actual Xe", ylabel="Predicted Xe")
    else:
        axes[row, 1].set(xlabel="Actual Te", ylabel="Predicted Te")

    axes[row, 1].legend(loc="upper right")
            
    pred += 1
    succ += 1

r2 = np.round(r2_score(truths, outputs), 3)

if train_param == "xe":
    axes[0, 0].set_title(
        "Residual plots of predicted and actual eutectic proportion Xe on SiamNN\nMAE: {}".format(mae))
    axes[0, 1].set_title(
        "Scatter plots of predicted vs actual eutectic proportion Xe on SiamNN\nR^2: {}".format(r2))
else: 
    axes[0, 0].set_title(
        "Residual plots of predicted and actual eutectic temperature Te on SiamNN\nMAE: {}".format(mae))
    axes[0, 1].set_title(
        "Scatter plots of predicted vs actual eutectic temperature Te on SiamNN\nR^2: {}".format(r2))

plt.show()

In [ ]:
l = batchsize
numplots = int(len(simpoutputs) / l)
simpfig, axes = plt.subplots(numplots, 2)
simpfig.set_figheight(45)
simpfig.set_figwidth(15)

pred = 0
succ = 1
x = np.arange(l)
acc = np.linspace(0, 1, 10)
mae = np.round(simptestloss, 3)
r2 = 0
        
for row in range(numplots):        
    axes[row, 0].scatter(x, simpoutputs[pred*l:succ*l] - simptruths[pred*l:succ*l], c="red")
    axes[row, 0].plot(x, np.zeros((l,)), c="green", label="0 Point")
    axes[row, 0].set(xlabel="Data Points", ylabel="Residuals")
    axes[row, 0].legend(loc="upper right")

    axes[row, 1].scatter(simptruths[pred*l:succ*l], simpoutputs[pred*l:succ*l], c="green")
    axes[row, 1].plot(acc, acc, label="Accuracy Line")
    
    if train_param == "xe":
        axes[row, 1].set(xlabel="Actual Xe", ylabel="Predicted Xe")
    else:
        axes[row, 1].set(xlabel="Actual Te", ylabel="Predicted Te")

    axes[row, 1].legend(loc="upper right")
            
    pred += 1
    succ += 1

r2 = np.round(r2_score(simptruths, simpoutputs), 3)

if train_param == "xe":
    axes[0, 0].set_title(
        "Residual plots of predicted and actual eutectic proportion Xe on SimpNN\nMAE: {}".format(mae))
    axes[0, 1].set_title(
        "Scatter plots of predicted vs actual eutectic proportion Xe on SimpNN\nR^2: {}".format(r2))
else: 
    axes[0, 0].set_title(
        "Residual plots of predicted and actual eutectic temperature Te on SimpNN\nMAE: {}".format(mae))
    axes[0, 1].set_title(
        "Scatter plots of predicted vs actual eutectic temperature Te on SimpNN\nR^2: {}".format(r2))

plt.show()

In [ ]:
# Print the values from the last batch processed just for the user to see
if train_param == "xe":
    disp = pd.DataFrame({
        'f(A,B) + f(B,A)': outputs + invouts,
        'Truth': np.round(truths, 3),
        'Siamese Pred': np.round(outputs, 3),
        'Simple Pred': np.round(simpoutputs, 3)
    })
else:
    disp = pd.DataFrame({
        'Truth': np.round(truths, 3),
        'Siamese Pred': np.round(outputs, 3), 
        'Simple Pred': np.round(simpoutputs, 3)
    })
        
disp.style.set_properties(**{'width': '150px'})

In [ ]:
# lossfig.savefig('D:\\Research\\UConn_ML\\Plots\\Lossplots_8_17_22.png')
# siamfig.savefig('D:\\Research\\UConn_ML\\Plots\\SiamNNplots_8_17_22.png')
# simpfig.savefig('D:\\Research\\UConn_ML\\Plots\\SimpNNplots_8_17_22.png')